# Why Profiling is needed?

Profiling is important to understand which parts of program consume most of the resources. This helps to improve the slow executing code segments to perform better or to focus on finding alternatives. Not only CPU, but also memory(RAM), disk usage(Disk I/O), network operations etc. also can be measured to determine bottlenecks of a program.

The basic method of identifying the bottlenecks is understanding the time consumption of the program sections. In jupyter notebook we can use `%%timeit` magic, time.time() or time decorators. In order to test the mentioned techniques we will define a special function named `Julia Set` which is Heavy CPU bound and less memory consuming non linear time consuming task. More technically speaking this is a fractal function which generates a complex output image.


<center><image src="./img/1.jpg" width="200px" /></center>

The basic psuedo code for calculation is as follows. In here coordinates are imaginary numbers and max_iter is a predefined variable for the function.
<pre style='color:yellow'> 
coordinates = []

for z in coordinates:
    for _ in range(max_iter):
        
        if (abs(z)< thres):
            z = z*z + c
        else:
            break
</pre>

But for the sake of testing various scenarios following imlpementation has few other parts added to it.

In [1]:
import time
import cv2

# area of imaginary space to calculate pixel values
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_img = -0.62772, -.42193

In [64]:
def display_img(arr):
    '''
    Function to display the generated output as an image.
    '''
    import numpy as np
    arr = arr.reshape((int(len(arr)**0.5), int(len(arr)**0.5)), order='C')

    arr = np.array(arr, dtype=np.uint8)
    cv2.imshow("Julia set", arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [50]:
def calculate_juliaset_serial(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

In [51]:
def calc_juliaset_time(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs), to build Julia set"""

    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width

    x = []
    y = []

    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step

    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_img))
    
    print("Length of x:", len(x))
    print("Total elements:", len(zs))

    start_time = time.time()
    output = calculate_juliaset_serial(max_iterations, zs, cs)
    end_time = time.time()

    secs = end_time - start_time
    print(calculate_juliaset_serial.__name__ + " took", secs, "seconds")

    return output

In [53]:
# reasonable defaults for a laptop
val = calc_juliaset_time(desired_width=1000, max_iterations=300)
display_img(val)

Length of x: 1000
Total elements: 1000000
calculate_juliaset_serial took 6.129579544067383 seconds


As above we can use the julia set to as a baseline task to check the performance. In the above case we have used the good old print statement with time difference to measure the performance. But this time change with the other processes running in the computer. Also print statements like above causes inconvienience in the long run. Instead we can use a decorator to measure time and print. (Or in Jupyter notebooks magic functions :D )